In [ ]:
import keras

In [ ]:
#path = "data/work_data/"
#path = "data/work_data/sample/"
path = "data/base_data/"
full_set_mode = "base" in path
model_path = "model/"
batch_size=10

In [ ]:
import utils
from utils import *
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline
from __future__ import division, print_function
from keras.layers import Input, Dense
from keras.models import Model
from keras.preprocessing import image
import os
from glob import glob
import numpy as np
import annotationHandler; reload(annotationHandler)
import bboxgenerator; reload(bboxgenerator)

In [ ]:
#This is a workaround for now to get bounding-boxes from the filename-structure of the keras-impl.
#we have to reassign the batches afterwards with boundingboxes provided.
train_path = path + 'train'
valid_path = path + 'valid'
size=(500,500)
batches = get_batches(train_path, batch_size=batch_size, shuffle=False, target_size=size)
train_bboxes = annotationHandler.create_bbx("data/annotations", size, batches.filenames, train_path)
if not full_set_mode:
    val_batches = get_batches(valid_path, batch_size=batch_size, shuffle=False, target_size=size)
    val_bboxes = annotationHandler.create_bbx("data/annotations", size, val_batches.filenames, valid_path)

In [ ]:
batches = bboxgenerator.DirectoryIterator(directory=train_path, batch_size=batch_size, target_size=size, shuffle=True, mappable_extras=[train_bboxes])
if not full_set_mode:
    val_batches = bboxgenerator.DirectoryIterator(valid_path, batch_size=batch_size, target_size=size, shuffle=False, mappable_extras=[val_bboxes])

In [ ]:
from vgg16bn import Vgg16BN
vgg = Vgg16BN(size,include_top=False)
model = vgg.model

for layer in model.layers:
    layer.trainable=False

In [ ]:
p = 0.2
x = model.output
x = MaxPooling2D()(x)
x = BatchNormalization(axis=1)(x)
x = Dropout(p/4)(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(p)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(p/2)(x)
x_bb = Dense(4, name='bb')(x)
x_class = Dense(8, activation='softmax', name='class')(x)

In [ ]:
model = Model(input=model.input, output=[x_bb, x_class])
model.compile(Adam(lr=0.001), 
              loss=['mse', 'categorical_crossentropy'], 
              metrics=['accuracy'],
              loss_weights=[.001, 1.])

In [ ]:
def train(model, epochs, full_run):
    if not full_run:
        model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=epochs, 
                            validation_data=val_batches, nb_val_samples=val_batches.nb_sample)
    else:
        model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=epochs)

In [ ]:
train(model, 3, full_run=full_set_mode)

In [ ]:
train(model, 7, full_run=full_set_mode)

In [ ]:
train(model, 3, full_run=full_set_mode)

In [ ]:
model.save(path + 'results/500_bbox_model.keras')

## Submit

In [ ]:
test_batches = get_batches("data/base_data/test", batch_size=batch_size, shuffle=False, target_size=size, class_mode=None)

In [ ]:
preds = model.predict_generator(test_batches, test_batches.nb_sample)

In [ ]:
filenames = test_batches.filenames
ids = np.array([f[8:] for f in filenames])
classes = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']
def do_clip(arr, mx): return np.clip(arr, (1-mx)/7, mx)
subm = do_clip(preds[1],0.82)

In [ ]:
submission = pd.DataFrame(subm, columns=classes)
submission.insert(0, 'image', ids)
submission.head()

In [ ]:
submission.to_csv(path + 'results/fishy6.csv', index=False)

In [ ]:
from pprint import pprint
pprint(np.argmax(preds[0][0]))
pprint(preds[0][:5])

In [ ]:
test_sloth_format = annotationHandler.save_as_sloth_format(preds[0], size, filenames, "data/base_data/test")

In [ ]:
import json
with open(path + 'results/preds_bboxes.json', 'w') as outfile:
    json.dump(test_sloth_format, outfile)

In [ ]:
print(test_sloth_format)